Data Collection and Cleaning

In [17]:
import pandas as pd
import numpy as np
import yfinance as yf
import os 
import time
from scipy.stats import zscore

Data Collection (only)

In [18]:
# define the tickers symbols

# equities = ['AAPL', 'MSFT', 'GOOGL', 'SPY'] #list of equities to download apple, microsoft, google and S&P 500 ETF
fx_pairs = ['EURUSD=X', 'GBPUSD=X'] #only using Euro/USD and GBP/USD exchange rates

tickers = fx_pairs

In [19]:
#downloading the historical data from yahoo finance

data = yf.download(tickers, start='2020-10-03', end='2025-10-09', interval='1d', group_by='ticker', auto_adjust=True, threads=True, actions=True)

[*********************100%***********************]  2 of 2 completed


In [20]:
# to view the pulled data from the api 

print(data.head())
print(data.tail())

Ticker      EURUSD=X                                                 \
Price           Open      High       Low     Close Volume Dividends   
Date                                                                  
2020-10-05  1.172196  1.179663  1.171930  1.172044      0       0.0   
2020-10-06  1.178828  1.180735  1.176734  1.179245      0       0.0   
2020-10-07  1.173737  1.178139  1.172541  1.173764      0       0.0   
2020-10-08  1.176554  1.178134  1.173530  1.176747      0       0.0   
2020-10-09  1.176706  1.182452  1.175876  1.176700      0       0.0   

Ticker                   GBPUSD=X                                       \
Price      Stock Splits      Open      High       Low     Close Volume   
Date                                                                     
2020-10-05          0.0  1.292808  1.298752  1.290156  1.293109      0   
2020-10-06          0.0  1.299207  1.300898  1.292340  1.299258      0   
2020-10-07          0.0  1.288477  1.292925  1.285033  1.2883

In [21]:
import os

print(os.path.isdir("/Users/akilfiros/Desktop/Projects/Side Projects /Quant-Backtesting/Data"))


True


In [22]:
#saving the data to a csv file

data.reset_index(inplace=True) #by doing this reset_index() I am telling pandas to take whatever was previously acting as the index (Date) and move it into a proper column.
data.columns = [f"{col[0]}.{col[1]}" for col in data.columns]
data.to_csv("/Users/akilfiros/Desktop/Projects/Side Projects /Quant-Backtesting/Data/market_data_not_cleaned.csv",index=False,mode='w')

#what happens with index=False and mode='w' included in the above line is that now when you're running the notebook from the start again the pulled data from yfinance which is being saved into the csv file will not create a new csv file with the same name instead it rewrite the data in the csv file and only the data will be updated.
#the index=False makes sure that pandas does not create an extra unnamed 0 column when writing the CSV


Data Cleaning (only)

In [23]:
#loading the csv file created and then reading into it

df = pd.read_csv('/Users/akilfiros/Desktop/Projects/Side Projects /Quant-Backtesting/Data/market_data_not_cleaned.csv', header= 1)
print(df.head())

   2020-10-05  1.172195553779602  1.1796625852584839  1.1719304323196411  \
0  2020-10-06           1.178828            1.180735            1.176734   
1  2020-10-07           1.173737            1.178139            1.172541   
2  2020-10-08           1.176554            1.178134            1.173530   
3  2020-10-09           1.176706            1.182452            1.175876   
4  2020-10-12           1.181335            1.182732            1.178800   

   1.1720443964004517  0  0.0  0.0.1  1.292808175086975  1.298751950263977  \
0            1.179245  0  0.0    0.0           1.299207           1.300898   
1            1.173764  0  0.0    0.0           1.288477           1.292925   
2            1.176747  0  0.0    0.0           1.291656           1.296933   
3            1.176700  0  0.0    0.0           1.293929           1.301507   
4            1.181684  0  0.0    0.0           1.302932           1.307976   

   1.2901561260223389  1.2931090593338013  0.1  0.0.2  0.0.3  
0          

In [24]:
#handling/checking missing values
missing_values = df.isnull().sum().sum()
print("Missing values:",missing_values)

if missing_values >0:
    df_cleaned = df.dropna()
    print(f"Number of missing values: {missing_values}")
else :
    print("No missing values")

print(df.isnull().sum()) #verify that there is no missing values


Missing values: 0
No missing values
2020-10-05            0
1.172195553779602     0
1.1796625852584839    0
1.1719304323196411    0
1.1720443964004517    0
0                     0
0.0                   0
0.0.1                 0
1.292808175086975     0
1.298751950263977     0
1.2901561260223389    0
1.2931090593338013    0
0.1                   0
0.0.2                 0
0.0.3                 0
dtype: int64


In [25]:
#identify duplicates
duplicate_count = df.duplicated().sum() #this returns the number of duplicated rows
print("Duplicated rows: ", duplicate_count)
# if duplicated rows is 0 then that means there is no duplicated rows in the data.

if duplicate_count > 0:
    df=df.drop_duplicates()
    print(f"Dropped {duplicate_count} duplicate rows")
else:
    print("No duplicate rows found")


Duplicated rows:  0
No duplicate rows found


In [26]:
#handling outliers

numeric_cols = df.select_dtypes(include=[np.number]).columns
# """
# What it does: selects the names of columns in df whose dtype is numeric (int/float).
#
# Return value / type: a pandas.Index of column names (e.g. Index(['A', 'B', 'C'])).
#
# Why: you only want to compute z-scores on numeric data (prices, volumes, etc.), not on string columns like Ticker or Date.
#
# Pitfall: if there are no numeric columns this will be empty — later code will fail. Also ensure numeric columns are actually numeric (no stray strings).
# """

z_scores = np.abs(zscore(df[numeric_cols]))
# """
# What it does, step by step:
#
# 1. df[numeric_cols] selects a DataFrame of numeric columns.
#
# 2. zscore(...) (from scipy.stats) computes the z-score for each value = (x - mean_column) / std_column. By default axis=0 so the mean/std are computed per column.
#
# 3. np.abs(...) takes the absolute value of the z-scores so that very negative and very positive deviations are treated the same.
#
# Return value / type: z_scores is a NumPy array (not a DataFrame) with the same shape as df[numeric_cols] — i.e. (n_rows, n_numeric_cols).
#
# Notes:
#
# - zscore uses ddof=0 by default (population std). That affects the scale slightly vs pandas.std() default (which uses ddof=1).
#
# - If a column has zero variance (std = 0), zscore will produce NaN or inf. You should handle constant columns first.
# """

#defining the threshold
threshold = 3
# """
# sets the cutoff for flagging outliers. A z-score > 3 (or < -3) is a common “3-sigma” rule of thumb, meaning: values with absolute z-score above 3 are considered extreme relative to that column’s distribution.
# """

outlier_mask = (z_scores > threshold)
# """
# What it does: creates a boolean array with the same shape as z_scores, where True indicates that the absolute z-score for that cell exceeds the threshold.
#
# Type / shape: NumPy boolean array (n_rows, n_numeric_cols).
#
# Example :
# [[False, False, True],
#  [False, False, False],
#  [True, False, False]]
#  means row 0, col 2 is an outlier; row 2, col 0 is an outlier.
# """

outlier_rows = np.where(outlier_mask)[0]
# """
# What np.where(outlier_mask) returns: a tuple (rows, cols) of 1-D arrays of indices where outlier_mask is True.
#
# Taking [0]: selects the row indices of every flagged cell. So outlier_rows becomes a 1-D NumPy array containing row indices — one entry for each flagged cell. If the same row had outliers in multiple columns it will appear multiple times.
#
# Example: if outlier_mask has True at (0,2) and (2,0) then np.where(outlier_mask) is (array([0,2]), array([2,0])) and [0] gives array([0,2]).
# """

print(f"Potential outlier rows: {len(np.unique(outlier_rows))}")
# """
# prints the number of unique rows that contain at least one outlier, by taking np.unique of outlier_rows (so duplicates are removed), then taking the length.
#
# Why np.unique used here: because outlier_rows can list the same row multiple times (once per column that flagged it).
# """

if len(outlier_rows) > 0:
    df = df[(z_scores < threshold).all(axis=1)]
    print(f"Removed rows with extreme outliers")
else :
    print("No extreme outlier found")
# """
# Filtering df: df[(z_scores < threshold).all(axis=1)] :
# (z_scores < threshold) is a boolean array where True = cell z-score < threshold.
#
# .all(axis=1) reduces per-row and returns a 1-D boolean array of length n_rows where True means every numeric column in that row has |z| < threshold. In other words: keep rows where no numeric column exceeds threshold.
#
# df[...] uses that boolean vector to filter rows in the DataFrame; rows with any flagged outlier are dropped.
#
# Effect: you remove any row that had at least one numeric column with |z| ≥ threshold.
# """


Potential outlier rows: 4
Removed rows with extreme outliers


In [27]:
#Reset index and ensure numeric types

df.reset_index(drop=True, inplace=True)
df[numeric_cols] = df[numeric_cols].astype(float)

print("\n Data cleaning complete!")
print(df.info())


 Data cleaning complete!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   2020-10-05          0 non-null      object 
 1   1.172195553779602   0 non-null      float64
 2   1.1796625852584839  0 non-null      float64
 3   1.1719304323196411  0 non-null      float64
 4   1.1720443964004517  0 non-null      float64
 5   0                   0 non-null      float64
 6   0.0                 0 non-null      float64
 7   0.0.1               0 non-null      float64
 8   1.292808175086975   0 non-null      float64
 9   1.298751950263977   0 non-null      float64
 10  1.2901561260223389  0 non-null      float64
 11  1.2931090593338013  0 non-null      float64
 12  0.1                 0 non-null      float64
 13  0.0.2               0 non-null      float64
 14  0.0.3               0 non-null      float64
dtypes: float64(14), object(1)
memory usage: 132

In [28]:
#saving the data to a csv file

data.to_csv("/Users/akilfiros/Desktop/Projects/Side Projects /Quant-Backtesting/Data/market_data_cleaned.csv",index=False,mode='w')